In [ ]:
# Dynamic Pricing for Urban Parking Lots
# Capstone Project - Summer Analytics 2025
# Developed for Kaggle - Complete Implementation

import pandas as pd
import numpy as np
from math import radians, cos, sin, sqrt, atan2
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
import time

output_file("dynamic_prices.html")

# ------------------------- Load Data -------------------------
data = pd.read_csv("/kaggle/input/dataset-capstone-project/dataset capstone project.csv")

# Assume base price is 10 for all lots initially
data["BasePrice"] = 10.0

# Vehicle type weights
vehicle_weights = {"car": 1.0, "bike": 0.5, "truck": 1.5}
data["VehicleWeight"] = data["VehicleType"].map(vehicle_weights)

# Normalize necessary columns
data["OccRate"] = data["Occupancy"] / data["Capacity"]
data["NormQueue"] = data["QueueLength"] / data["QueueLength"].max()
data["NormTraffic"] = data["TrafficLevel"] / data["TrafficLevel"].max()

# ------------------------- Model 1: Baseline -------------------------
def baseline_price_update(price, occ_rate, alpha=1.5):
    return price + alpha * occ_rate

# ------------------------- Model 2: Demand-Based -------------------------
def demand_function(row, alpha=1.2, beta=1.5, gamma=1.0, delta=2.0, epsilon=1.0):
    return (
        alpha * row["OccRate"] +
        beta * row["NormQueue"] -
        gamma * row["NormTraffic"] +
        delta * row["IsSpecialDay"] +
        epsilon * row["VehicleWeight"]
    )

def demand_price_update(base_price, demand, lambda_=0.2):
    norm_demand = (demand - 1) / 4  # scale to ~[-0.25, 0.75]
    price = base_price * (1 + lambda_ * norm_demand)
    return np.clip(price, 5, 20)

# ------------------------- Model 3: Competitive Pricing -------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

def compute_competition_adjustment(df, lot_id, lat, lon, current_price):
    nearby = df[(df["LotID"] != lot_id)].copy()
    nearby["Distance"] = nearby.apply(lambda r: haversine(lat, lon, r["Latitude"], r["Longitude"]), axis=1)
    close = nearby[nearby["Distance"] <= 1.0]  # within 1 km
    if close.empty:
        return current_price
    avg_price = close["DynamicPrice"].mean()
    if current_price > avg_price and df.loc[lot_id, "Occupancy"] >= df.loc[lot_id, "Capacity"]:
        return max(current_price - 1, 5)  # encourage reroute
    elif current_price < avg_price:
        return min(current_price + 1, 20)  # increase for advantage
    else:
        return current_price

# ------------------------- Real-Time Simulation -------------------------
def simulate_dynamic_pricing(df):
    df = df.copy()
    df["DynamicPrice"] = df["BasePrice"]
    df["TimeSlot"] = pd.to_datetime(df["Timestamp"])

    # Simulate pricing step-by-step
    time_slots = sorted(df["TimeSlot"].unique())
    lot_ids = df["LotID"].unique()

    # Visualization setup
    prices_over_time = {lot: [] for lot in lot_ids}
    source = ColumnDataSource({"x": [], **{str(l): [] for l in lot_ids}})
    p = figure(title="Dynamic Prices Over Time", x_axis_type="datetime", width=800, height=400)
    lines = {l: p.line(x="x", y=str(l), source=source, legend_label=f"Lot {l}") for l in lot_ids}
    p.legend.click_policy = "hide"

    for t in time_slots:
        batch = df[df["TimeSlot"] == t]

        for i, row in batch.iterrows():
            dem = demand_function(row)
            new_price = demand_price_update(row["BasePrice"], dem)
            df.at[i, "DynamicPrice"] = new_price

        # Competitive adjustment
        for i, row in batch.iterrows():
            new_price = compute_competition_adjustment(df, row["LotID"], row["Latitude"], row["Longitude"], row["DynamicPrice"])
            df.at[i, "DynamicPrice"] = new_price
            prices_over_time[row["LotID"]].append(new_price)

        # Update plot data
        new_data = {"x": [t]}
        for lot in lot_ids:
            new_data[str(lot)] = [prices_over_time[lot][-1] if prices_over_time[lot] else 10]
        source.stream(new_data)

    show(p)
    return df

# ------------------------- Run Simulation -------------------------
final_df = simulate_dynamic_pricing(data)
final_df.to_csv("final_dynamic_pricing_output.csv", index=False)
print("Simulation complete. Output saved.")
